In [1]:
%matplotlib inline

import datetime
import json
import os
import matplotlib
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import pandas.io.formats.excel
import re
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

In [2]:
banks_list = ['АО «Кредит Европа Банк»',
              'ПАО «АК БАРС» БАНК',
              'АО «Райффайзенбанк»',
              'АО Газпромбанк',
              'ПАО РОСБАНК',
              'ПАО "СКБ-БАНК"',
              'ПАО «СОВКОМБАНК»',
              'Альфабанк',
              'РНКБ Банк (ПАО)',
              'Хоумкредитбанк',
              'ПАО «ПОЧТА БАНК»',
              'ПАО Банк «Финансовая Корпорация Открытие»',
              'ПАО «ПРОМСВЯЗЬБАНК»',
              'БАНК ВТБ (ПАО)',
              'АО «ТИНЬКОФФ БАНК»',
              'АКБ «Авангард» - ПАО',
              'АО «ОТП БАНК»',
              'АО «Банк Русский Стандарт»',
              'ПАО «БАНК УРАЛСИБ»',
              'АКБ «АБСОЛЮТ БАНК» (ПАО)',
              'ПАО КБ «Центр-инвест»',
              'ПАО «МИНБАНК»',
              'ПАО «МосОблБанк»',
              'ПАО КБ "УБРИР"',
              'АКЦИОНЕРНОЕ ОБЩЕСТВО БАНК "СЕВЕРНЫЙ МОРСКОЙ ПУТЬ"',
              'ПАО Банк «Возрождение»',
              'ПАО «Банк «Санкт-Петербург»',
              'ПАО «МТС-БАНК»',
              'ПАО «ЗАПСИБКОМБАНК»',
              'КБ «РЕНЕССАНС КРЕДИТ»  ООО',
              'ПАО АКБ "СВЯЗЬ-БАНК"',
              'АО «Юникредит Банк»',
              'Банк «Зенит» (ПАО)', 
              'АО «Акционерный Банк "Россия»', 
              'АКЦИОНЕРНОЕ ОБЩЕСТВО «ВСЕРОССИЙСКИЙ БАНК РАЗВИТИЯ РЕГИОНОВ»', 
              'АО «Россельхозбанк»',
              'ПАО  "СБЕРБАНК РОССИИ"',
              'ПАО «Азиатско-Тихоокеанский банк»',
              'Киви Банк АО'
             ]

In [3]:
bnames = pd.read_excel('bank_names.xlsx',header=None)
dfz = pd.read_csv('01.05-31.05.csv')
#[for each in dfz['info_system']]
bnames[1] = bnames[1].astype(str)
#dfz = pd.read_csv('01.01-31.01.csv')
#dfz['bank'] = [bnames[bnames['testroiv08'] == name]['тест'] for name in dfz['info_system']]
#dfz
bnames[bnames[1] == '774207'].reset_index()[0][0]
bnames[0][0]

'тест'

In [4]:
success_status = ['200',
 '200.0',
 '202',
 '202.0']
#dfz = pd.read_excel('01.12-31.12v4.xls')
#bnames = pd.read_excel('bank_names.xlsx',header=None)
dfz = pd.read_csv('ppl01.07-31.07_out.csv')
dfz['bank'] = [bnames[bnames[1] == str(name)][1].unique().item() for name in dfz['info_system']]
#dfz.loc[dfz['info_system'] == '774207']
banks_list2 = dfz['bank'].unique().tolist()
dfz['success'] = dfz['success'].astype('str')
# df = df.loc[~(df['bank'] == '776426_3S')]
#dfz['init_date_time'] = [datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in dfz['init_date_time'].tolist()]
dfz['init_date_time'] = pd.to_datetime(dfz['init_date_time'])
bank_names_df = pd.read_excel('bank_names.xlsx', header=None)
bank_names_df[1] = bank_names_df[1].astype(str)

def is_succeed(df,user):
    tmp_df = df.loc[df['user_id'] == user]
    if tmp_df.loc[tmp_df['success'].isin(success_status)].shape[0] > 0:
        return True
    else:
        return False

def num_tries(df, user):
    tmp_df = df.loc[df['user_id'] == user]
    if tmp_df.loc[~tmp_df['success'].isin(success_status)].shape[0] > 0:
        return tmp_df.shape[0]
    else:
        return 1

def num_days(df, user):
    tmp_df = df.loc[dfz['user_id'] == user]
    fail_df = tmp_df.loc[~tmp_df['success'].isin(success_status)]
    success_df = tmp_df.loc[tmp_df['success'].isin(success_status)]
    if fail_df.shape[0] > 0 and success_df.shape[0] > 0:
        fail_day = min(fail_df['init_date_time'])
        
        if max(success_df['init_date_time']) < fail_day:
            print('failed on user: {}'.format(user))
            return 0
        
        for date in success_df['init_date_time']:
            if date >= fail_day:
                success_day = date
                return (success_day - fail_day).days
            else:
                pass           
                
    elif fail_df.shape[0] == 0:
        return 0
    else:
        fail_day1 = min(fail_df['init_date_time'])
        fail_day2 = max(fail_df['init_date_time'])
        return (fail_day2 - fail_day1).days

def bank_stat(df):
    all_users = []
    for user in set(df['user_id']):
        tmp_dict = {}
        tmp_dict['user'] = user

        tmp_dict['success'] = is_succeed(df, user)
        tmp_dict['num_tries'] = num_tries(df, user)
        tmp_dict['num_days'] = num_days(df, user)
        all_users.append(tmp_dict)
    
    return all_users


In [7]:
# last_df = pd.DataFrame(all_users)

def prep_df(df, success = False):
    if success == 'with_fails':
        out_df = df.loc[(df['num_tries'] > 1) & (df['success'] == False)].copy()
        tries = out_df['num_tries'].value_counts()
        out_df['tries_aggr'] = [tries[each] for each in out_df['num_tries']]
    elif success == 'with_success':
        out_df = df.loc[(df['num_tries'] > 1) & (df['success'] == True)].copy()
        tries = out_df['num_tries'].value_counts()
        out_df['tries_aggr'] = [tries[each] for each in out_df['num_tries']]
    elif success == 'both':
        out_df = df.loc[(df['num_tries'] > 1)].copy()
        tries = out_df['num_tries'].value_counts()
        out_df['tries_aggr'] = [tries[each] for each in out_df['num_tries']]
    elif success == 'every':
        out_df = df.copy()
        tries = out_df['num_tries'].value_counts()
        out_df['tries_aggr'] = [tries[each] for each in out_df['num_tries']]
    elif isinstance(success, bool):
        out_df = df[df['success'] == success].copy()
        tries = out_df['num_tries'].value_counts()
        out_df['tries_aggr'] = [tries[each] for each in out_df['num_tries']]
    else:
        raise TypeError("Success should be bool type or str ['with_fail', 'with_success', 'both']")
    
    return out_df
    
# out = prep_df(last_df, success = False)
# #print(out.reset_index())
# print(out.shape)
# ab = out['num_days'].value_counts()
# sns.distplot(out['num_tries'], kde=False, rug=False, hist_kws={"rwidth":0.90}).set(xlabel='Распределение количества попыток регистраций c ошибкой, где повторной регистрации не было или закончилась провалом')
# #sns.distplot(out['num_tries'], kde=False, rug=False, hist_kws={"rwidth":0.90}).set(xlabel='Распределение количества попыток регистраций c ошибкой, где повторная регистрация закончилась успехом')
# #sns.jointplot(x=out['num_tries'], y=out['num_days'], data=out.loc[:, ['num_days', 'num_tries']])

def stats(dfa, bank_name):
    all_users = bank_stat(dfa)
    last_df = pd.DataFrame(all_users)
    success_bank_out = prep_df(last_df, success = True)
    people_come = len(all_users)
    success_people = success_bank_out.shape[0]
    try:
        success_first_try = success_bank_out['num_tries'].value_counts()[1]
    except:
        success_first_try = 0
    success_multiple_try = success_people - success_first_try
    # success_multiple_try_1_day = success_bank_out['num_days'].value_counts()[0]
    fail_bank_out = prep_df(last_df, success = False)
    fails_people = fail_bank_out.shape[0]
    try:
        fail_first_try = fail_bank_out['num_tries'].value_counts()[1]
    except:
        fail_first_try = 0
    fail_multiple_try = fails_people - fail_first_try
    # fail_multiple_try_1_day = fail_bank_out['num_days'].value_counts()[0]
    names = ['Банк', 'Человек', 'Успешно всего', 'Успешно с 1 попытки', 'Успешно с нескольких попыток', 'Неуспешно всего', 'Неуспешно с 1 попытки', 'Неуспешно с нескольких попыток']
    
    n = bank_names_df[bank_names_df[1] == str(dfa['info_system'].reset_index()['info_system'][0])]
    
    return dict(zip(names, [n.reset_index()[0][0], people_come, success_people, success_first_try, success_multiple_try, fails_people, fail_first_try, fail_multiple_try]))

writer = pd.ExcelWriter('final_peaple_01053105.xlsx', engine = 'xlsxwriter')

for i in [1]:
    #mask = [True if date.month == i else False for date in dfz['init_date_time']]
    #df = dfz.loc[mask]
    df = dfz
    names = ['Банк', 'Человек', 'Успешно всего', 'Успешно с 1 попытки', 'Успешно с нескольких попыток', 'Неуспешно всего', 'Неуспешно с 1 попытки', 'Неуспешно с нескольких попыток']
    final_df = pd.DataFrame(columns=names)
    for bank in banks_list2:
        tmp_df = df.loc[df['bank'] == bank]
        if tmp_df.shape[0] > 0:
            stat = stats(tmp_df, bank)
            final_df = final_df.append(stat, ignore_index=True)
        else:
            print(bank)        
    final_df.to_excel(writer, sheet_name = '{}'.format(i))

writer.save()
writer.close()

failed on user: 1019830988


In [6]:
final_df

,Банк,Человек,Успешно всего,Успешно с 1 попытки,Успешно с нескольких попыток,Неуспешно всего,Неуспешно с 1 попытки,Неуспешно с нескольких попыток
0,"ПАО ""СБЕРБАНК РОССИИ""",118,113,108,5,5,1,4
1,БАНК ВТБ (ПАО),133,131,131,0,2,2,0
2,РНКБ БАНК (ПАО),9,9,9,0,0,0,0
3,"ПАО ""Московский областной банк""",6,6,6,0,0,0,0
4,"АО ""Генбанк""",5,5,5,0,0,0,0
5,АО «Альфа-Банк»,48,48,48,0,0,0,0
6,АО КБ «Ситибанк»,2,2,2,0,0,0,0
7,"ПАО Банк ""Финансовая Корпорация Открытие""",6,6,6,0,0,0,0
8,"ПАО ""ПРОМСВЯЗЬБАНК""",1,1,1,0,0,0,0
9,"ООО""КАМКОМБАНК""",4,4,4,0,0,0,0


In [6]:
dfc = pd.read_csv('06.07-12.07.csv')
users = dfc['user_id'].unique()


dfc['success'] = dfc['success'].astype(int)
u = []
for user in users:
    useries = dfc[dfc['user_id'] == user]['success'].tolist()
    if 200 in useries or 202 in useries:
        pass
    else:
        u.append(user)
print()


returns = [us for us in u if us in dfz['user_id'].unique()]
print(f'Не сдали на прошлой неделе {len(u)} чел., вернулись {len(returns)} чел.')
for r in returns:
    print(dfz[dfz['user_id'] == r]['success'].tolist())
    



Не сдали на прошлой неделе 7 чел., вернулись 1 чел.
['202.0', '202.0', '202.0']


In [7]:
dfc

,Unnamed: 0,request_id,init_date_time,smev_exit_code_description,bkk_sound_status_code,bkk_photo_status_code,success,info_system,k2_1,service_centre,...,n1_2_1,n1_2_2,n1_2_3,n2,n3,n6,n7,n8,n10,bank
0,0,00c9a777-b1f9-11ea-8e17-0050568896ec,2020-06-19 06:49:27,Регистрация прошла успешно.,NaN,NaN,200.0,771601_3S,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.095940e+09,...,4.294967e+09,4.294967e+09,4.294967e+09,0.0,0.0,0.0,0.0,0.0,0.0,АО «Россельхозбанк»
1,1,00e9e2ba-afe2-11ea-ba22-0173ce901b1b,2020-06-16 14:59:50,Регистрация прошла успешно.,NaN,NaN,200.0,774207,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.089458e+09,...,4.294967e+09,4.294967e+09,4.294967e+09,0.0,0.0,0.0,0.0,0.0,0.0,БАНК ВТБ (ПАО)
2,2,0148f812-b13e-11ea-9238-fc65ebe9e4ff,2020-06-18 08:31:12,Регистрация прошла успешно.,NaN,NaN,200.0,774207,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.099592e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,БАНК ВТБ (ПАО)
3,3,01ab7130-afac-11ea-9c24-0050569e4871,2020-06-16 08:33:20,Регистрация прошла успешно.,NaN,NaN,200.0,776426_3S,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.087632e+09,...,1.000000e+00,1.000000e+00,2.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,"ПАО ""СБЕРБАНК РОССИИ"""
4,4,0210f5b4-b0b4-11ea-b435-fc65ebe9e4ff,2020-06-17 16:03:09,Регистрация прошла успешно.,NaN,NaN,200.0,774207,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.099589e+09,...,4.294967e+09,4.294967e+09,4.294967e+09,0.0,0.0,0.0,0.0,0.0,0.0,БАНК ВТБ (ПАО)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,826,ff1f9ab4-b068-11ea-9ee0-9fc7ae1d3cf7,2020-06-17 07:06:05,Регистрация прошла успешно.,NaN,NaN,200.0,775101_3S,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.081509e+09,...,4.294967e+09,4.294967e+09,4.294967e+09,0.0,0.0,0.0,0.0,0.0,0.0,"ПАО ""МТС-БАНК"""
827,827,ff5cf3e0-afd8-11ea-af41-0050569eba89,2020-06-16 13:55:26,Регистрация прошла успешно.,NaN,NaN,200.0,776426_3S,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.087637e+09,...,1.000000e+00,1.000000e+00,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,"ПАО ""СБЕРБАНК РОССИИ"""
828,828,ffb5be69-b07e-11ea-95d4-53f4c0e9ef0a,2020-06-17 09:43:35,Регистрация прошла успешно.,NaN,NaN,200.0,780301_3S,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.093124e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,"АО ""Акционерный Банк ""Россия"""
829,829,ffebeaa9-afd0-11ea-a92a-0173ce901b1b,2020-06-16 12:58:08,Регистрация прошла успешно.,NaN,NaN,200.0,774207,"[""[sound, CRT_sound_1.0.3, true, null]"",""[soun...",1.099589e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,БАНК ВТБ (ПАО)
